### Project - Radar Black Friday

Projeto que tem como principal objetivo coletar diariamente os preços de produtos vendidos pelas principais lojas ecommerce do Brasil e acompanhar se realmente existe uma redução de preço no famoso evento Black Friday.

#### Purpose of this notebook
Neste notebook você encontrará a etapa de coleta dos links que serão raspados.<br>

>Este notebook não tem fins educativos, portanto não será explicado o conceito dos métodos ou funções, somente o objetivo das etapas.


In [ ]:
# IMPORTAÇÃO DAS LIBS

import sqlite3 # -- SQLite 3 - Banco de Dados
from urllib.request import urlopen, Request #URL Lib para acessar link que será feito o Scraping
import bs4 #BeautifulSoup para analisar página HTML
import pandas as pd #Pandas para analisar os dados e armazenar em DataFrma
import datetime

#### Step 1 - Funções utilizadas

In [ ]:
#Funções de LOG

# Função para gravar o LOG de Execução da Etapa
def log_exec(state):
    try:
        query = "INSERT INTO log_exec (dt_reg, ds_step) Values (current_timestamp, '%s')"
        cursor.execute(query %(state))
        cnn.commit()
        return "Log Execução Gravado com Sucesso"
    except:
        return "Erro gravar Log Execução"
    
    
# Função para gravar o LOG do resultado da raspagem
def log_scraping(link, state, qt_precos, cursor):
    try:
        query = """
                INSERT INTO log_scraping 
                    (dt_insert, link_scraping, status_exec, qt_prices) 
                    VALUES (current_timestamp, '%s', '%s', %s)
                """
        cursor.execute(query %(link, state, qt_precos))
        cnn.commit()
        return "Log Scraping Gravado com Sucesso"
    except:
        return "Erro gravar Log Scraping"   
    
    
# Função para gravar o LOG da importação no banco de dados
def log_insert(dt_inicio,dt_final, qt_sucesso, qt_falha):
    try:
        query = """
                INSERT INTO log_insert_db 
                    (dt_start, dt_end, qt_success, qt_failure)
                    Values ('%s', '%s', %s, %s)
                """
        cursor.execute(query %(dt_inicio,dt_final, qt_sucesso, qt_falha ))
        cnn.commit()
        return "Log Insert Gravado com Sucesso"
    except:
        return "Erro Gravar Log Insert"        

In [ ]:
# Função para inserir os resultados no Banco de Dados

def inserirRegistros(list_result):
    
    #Contatores
    sucess = 0
    failure = 0
    
    dt_inicio = datetime.datetime.today()
    
    query = """
            INSERT INTO scraping_results
            (dt_insert, ds_category, ds_subcategory, ds_product, ds_ecommerce, price)
            VALUES (current_timestamp, '%s', '%s', '%s', '%s', %s) 
            """
    # Loop para importar todas as linhas do DataFrame
    for index, row in list_result.iterrows():
        try:
            cursor.execute(query %(row.categoria, row.subcategoria, row.nome_produto, row.loja, row.valor ))
            sucess += 1
        except:
            failure += 1
    cnn.commit()
    
    dt_final = datetime.datetime.today()
    log_insert(dt_inicio,dt_final, sucess, failure)

In [ ]:
# Função para ajudar a Tratar o Preço retirando o R$ e trocando a vírgula por ponto.

def limparPreco(preco):
    preco = preco.replace('R$ ', '').replace('.', '').replace(',', '.')
    return preco

In [ ]:
# Função para ler o arquivo CSV que contém a lista de links que serão raspados.

def listaLinks(file):
    list_links = pd.read_csv(file)
    return list_links

In [ ]:
# Função responsável por navegar todos os links e realizar a raspagem.

def coletarPrecos(list_links, cursor):
    list_produtos = []
    list_erros = [] 
    
    #contador
    erros = 0
    corretos = 0
    total = len(list_links)
    
    for index, row in list_links.iterrows():
        
        print('Erros: %s/%s' %(erros, total))
        print('Raspados: %s/%s' %((erros + corretos), total))
        print('Link sendo consultado: %s' %(row.link))
        
        try:
            corretos += 1
            
            #Abrir e realizar o BS4
            data = urlopen(row.link).read()
            soup = bs4.BeautifulSoup(data, features="html.parser")

            #Bs4 para localizar as Tags
            bs4_nomes = soup.findAll('h1', {'class' : 'OverviewArea_TitleText__1s_GP'})
            bs4_marcas = soup.findAll('img', {'class' : 'MerchantBrand_BrandImage__QoYbM'})
            bs4_precos = soup.findAll('a', {'class' : 'PriceBox_Value__2VuFN'})

            #Variavel que define o nome do Produto
            nome_produto = bs4_nomes[0].text              
            print("Produto sendo consultado: %s" %(nome_produto))
            
            log_scraping(row.link, 'Success', len(bs4_precos), cursor)
            
            for item in range(len(bs4_precos)):
            
                if item == 0:
                    print('Loja: %s | Preço: %s' %('_menor_preco_produto', bs4_precos[item].text))
                    list_produtos.append([row.categoria, row.subcategoria, nome_produto,'_menor_preco_produto', bs4_precos[item].text, row.link, datetime.datetime.today()])
                else:
                    print('Loja: %s | Preço: %s' %(bs4_marcas[item-1].get('title'), bs4_precos[item].text))
                    list_produtos.append([row.categoria, row.subcategoria, nome_produto,bs4_marcas[item-1].get('title'),bs4_precos[item].text, row.link, datetime.datetime.today()]) 
        except:
            erros += 1
            list_erros.append([row.link,row.categoria,row.subcategoria])    
            log_scraping(row.link, 'Failure', 0, cursor)

    return list_produtos

In [ ]:
# Função para transformar a lista de resultados em um Dataframe

def transformarResultado(list_result):
    df = pd.DataFrame(list_result, columns=['categoria','subcategoria','nome_produto', 'loja', 'valor', 'link', 'dt_insert'])
    df['valor'] = df['valor'].apply(limparPreco)
    df['valor'] = pd.to_numeric(df['valor'])
    return df

#### Step 2 - Processo de Scraping


In [ ]:
# Conexão com banco de dados radarBlackFriday
cnn = sqlite3.connect("radarBlackFriday.db")
cursor = cnn.cursor()

In [ ]:
#Registrar Log Inicio do Processo
log_exec('Start APP')

#Carregar arquivo CSV com lista dos Links
lista = listaLinks('list_links_20.csv')
log_exec('Lista de Links Carregada com %s links' %(len(lista)))

In [ ]:
log_exec('Inicio Scraping') #Registrar LOG Inicio do Scraping

result = coletarPrecos(lista , cursor) #Scraping
result = transformarResultado(result) #Transformar Resultado em DF

log_exec('Final Scraping') #Registrar LOG Final do Scraping

In [ ]:
log_exec('Inicio Armazenamento') #Registrar LOG Inicio da importação do resultado

inserirRegistros(result) # Importação do Resultado no SQLite

log_exec('Final Armazenamento') #Registrar LOG Final da importação do resultado

In [ ]:
#Finaliza processo e fecha a conexão
log_exec('Stop APP')
cnn.close